In [1]:
import os
from datasets import load_dataset 
import os
from datasets import load_from_disk, load_dataset
import PIL
import warnings
warnings.filterwarnings('ignore')
from torchvision import transforms
from datasets import load_dataset
from transformers import AutoImageProcessor
from pytorch_grad_cam import run_dff_on_image, GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional

In [2]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [3]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

In [4]:

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [5]:
model_checkpoint = "/workspace/model/hugginface_image_classifcation/new_hipjoint/resnet-18_20231218_1843/best.hf" # pre-trained model from which to fine-tune
batch_size = 32 # batch size for training and evaluation
dataset = load_from_disk('new_hipjoint_hfdataset')

In [6]:
val_ds = dataset['test']

In [7]:
from torchvision import transforms

image = dataset["test"]["image"][5]

In [13]:
image = image.convert('RGB')

In [8]:
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

In [9]:
trans = transforms.Compose([transforms.ToTensor(),
                    transforms.Resize((256, 256)),
                    transforms.Normalize(mean=mean, std=std)])

In [138]:
resnet_model = AutoModelForImageClassification.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/resnet-18_20231218_1843/best.hf")
convnext_model = AutoModelForImageClassification.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/convnext-large-224_20231218_1956/best.hf")
swin_model = AutoModelForImageClassification.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/swin-large-patch4-window12-384-in22k_20231218_2321/best.hf")
vit_model = AutoModelForImageClassification.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/vit-base-patch16-224_20231218_1404/best.hf")
swinv2_model = AutoModelForImageClassification.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/swin-large-patch4-window12-384-in22k_20231218_2321/best.hf")


processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [139]:
swin_processor = AutoImageProcessor.from_pretrained("/workspace/model/hugginface_image_classifcation/new_hipjoint/swin-large-patch4-window12-384-in22k_20231218_2321/best.hf")

In [140]:
inputs = processor(image, return_tensors="pt")
swin_inputs = swin_processor(image, return_tensors='pt')

In [141]:
# 중간 layer의 output을 저장하기 위한 함수
outputs = []
def hook(module, input, output):
    outputs.append(output)

# 마지막 hidden layer에 hook 등록
resnet_model.classifier[0].register_forward_hook(hook)
convnext_model.convnext.layernorm.register_forward_hook(hook)
swin_model.swin.pooler.register_forward_hook(hook)
vit_model.vit.layernorm.register_forward_hook(hook)


In [142]:
with torch.no_grad():
    output = resnet_model(**inputs)
    output = convnext_model(**inputs)
    output = swin_model(**swin_inputs)
    output = vit_model(**inputs)

In [143]:
for i in outputs:
    print(i.shape)

torch.Size([1, 512])
torch.Size([1, 1536])
torch.Size([1, 1536, 1])
torch.Size([1, 197, 768])


In [127]:
vit_model.classifier

Linear(in_features=768, out_features=7, bias=True)